In [1]:
###### import numpy as np
from data_iterator import DataIterator,prepare_data
import tensorflow as tf
# from model import *
import time
import random
import sys
# from utils import *
import tensorflow as tf
import tensorflow.keras.layers as layers
# import tensorflow.
import numpy as np

In [2]:
EMBEDDING_DIM = 18
HIDDEN_SIZE = 18 * 2
ATTENTION_SIZE = 18 * 2
best_auc = 0.0


In [3]:
train_file = "./data/local_train_splitByUser"
test_file = "./data/local_test_splitByUser"
uid_voc = "./data/uid_voc.pkl"
mid_voc = "./data/mid_voc.pkl"
cat_voc = "./data/cat_voc.pkl"
batch_size = 128
maxlen = 100
test_iter = 100
save_iter = 100
model_type = 'DNN'
seed = 2

In [4]:
model_path = "dnn_save_path/ckpt_noshuff" + model_type + str(seed)
best_model_path = "dnn_best_model/ckpt_noshuff" + model_type + str(seed)
#     gpu_options = tf.GPUOptions(allow_growth=True)
train_data = DataIterator(train_file, uid_voc, mid_voc, cat_voc, batch_size, maxlen, shuffle_each_epoch=False)
test_data = DataIterator(test_file, uid_voc, mid_voc, cat_voc, batch_size, maxlen)
n_uid, n_mid, n_cat = train_data.get_n()    

In [5]:
for src, tgt in train_data:
    uids, mids, cats, mid_his, cat_his, mid_mask, target, sl, noclk_mids, noclk_cats = prepare_data(src, tgt, maxlen, return_neg=True)
    break

In [6]:
mid_mask.shape,noclk_mids[:,:,1].shape, noclk_cats.shape

((128, 100), (128, 100), (128, 100, 5))

# 模型

In [18]:
class dice_layer(layers.Layer):
    def __init__(self,axis=-1, epsilon = 1e-10):
#         super(dice_layer, self).__init__()
        super().__init__()
        self.axis = axis
        self.epsilon = epsilon
        
    def build(self, input_shape):
        rand = tf.random_normal_initializer()(shape=[input_shape[-1]])
        self.alpha = tf.Variable(rand, dtype=tf.float32, name="alpha")
    def call(self, x):
        input_shape = list(x.shape)
        
        # 需要进行reduce计算的轴
        reducetion_axes = list(range(len(input_shape)))  # [0,1]
        del reducetion_axes[self.axis]      # [0]
        
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]  # [1,72]
        # 计算每个样本的某个特征f的均值        
        mean = tf.reduce_mean(x, axis= reducetion_axes) #(f,) 一共f个特征
        brodcast_mean = tf.reshape(mean, broadcast_shape) # reshape (1,f)
        
        # 计算方差
        std = tf.reduce_mean(tf.square(x-brodcast_mean) + self.epsilon, axis=reducetion_axes)
        std = tf.sqrt(std)
        
        # 还原
        brodcast_std = tf.reshape(std,broadcast_shape)
        
        # 标准化，_x的shape不变
        x_normal = (x-brodcast_mean)/ (brodcast_std + self.epsilon)
        
        # 以上过程即为BatchNormalization：
        # x_normed = layer.BatchNormalization(center=False, scale=False)(_x)
        
        # 标准化后使用 sigmoid 函数得到 x_p
        x_p = tf.sigmoid(x_normal)
        return self.alpha * (1.0 - x_p) * x + x_p * x

In [33]:
class fc_layer(tf.keras.layers.Layer):
    def __init__(self, fc_config=[128,64,1]):
        super(fc_layer, self).__init__()
        self.fc_layers = []
        self.dice_layers = []      
        for s in fc_config[:-1]:
            self.fc_layers.append(layers.Dense(s, name='dense'+str(s)))
            
        for s in fc_config[:-1]:
            self.dice_layers.append(dice_layer())

        self.fc_layers.append(layers.Dense( fc_config[-1]))
        
    def call(self, inp):
        fc_out = inp
        for i, _ in enumerate(self.fc_layers[:-1]):
                fc_out = self.dice_layers[i](self.fc_layers[i](fc_out))
        fc_out =  self.fc_layers[-1](fc_out)       
        return fc_out
#         self.y_hat = tf.nn.softmax(dnn3) + 0.00000001

In [34]:
class din_attention(tf.keras.layers.Layer):
    def __init__(self):
        super(din_attention, self).__init__()
        self.fc_part = fc_layer([80,40,1])
        
    def call(self, item_emb, hist_item_emb,item_mask,softmax_stag=True, mode='SUM'):
        """
        item_emb: (B,D)
        hist_item_emb: (B,T,D)
        item_mask: (B,T)
        此处的D均表示cat_embedding维度+item_embedding维度
        """
        masks = tf.math.equal(item_mask, tf.ones_like(item_mask)) # (B,T)
        item_size = item_emb.shape[-1]  
        item_hist_size = hist_item_emb.shape[-1] 
        # 为了方便计算相似度，将item复制至个数与hist_item长度相同
        queries = tf.tile(item_emb, [1,hist_item_emb.shape[1]]) #(B, D *T)
        queries = tf.reshape(queries, [-1,hist_item_emb.shape[1],hist_item_emb.shape[2]])  # (B,T,D)
        din_all = tf.concat([queries, hist_item_emb,queries-hist_item_emb, queries*hist_item_emb],axis=-1)  #(B,T,D*4)
        # 相似度得分
        similar_scores = self.fc_part(din_all) # (B,T,1)
        similar_scores = tf.reshape(similar_scores, [-1,1,hist_item_emb.shape[1]])  # (B,1,T)
#         print(din_all.shape,'scores',similar_scores.shape)
        
        # tf.where 根据mask将得分与hist_item_emb对应的位置匹配填充，不存在的地方，填充为极大值。
        key_masks = tf.expand_dims(masks, 1) # (B, 1, T)
        # 计分器容器
        paddings = tf.ones_like(similar_scores) * (-2 ** 32 + 1)
        
        scores = tf.where(key_masks, similar_scores, paddings) # (B, 1, T)
        
        if softmax_stag:
            scores = tf.nn.softmax(scores) # (B,1,T)
        scores = tf.reshape(scores,[-1,hist_item_emb.shape[1],1])    
        return scores

In [62]:
class auxiliary_net(tf.keras.layers.Layer):
    def __init__(self, auxi_config = [100,50,2] ):
        super().__init__() 
        self.bh_nor = layers.BatchNormalization()
        self.auxi_layers = []
        for n in auxi_config:
            self.auxi_layers.append(layers.Dense(n,activation='sigmoid',name='auxiliary_net'+str(n)))
    def call(self,inp):
#         print('inp',inp)
        dnn_res = self.bh_nor(inp, training=True)
#         dnn_res=inp
        for i in range(len(self.auxi_layers)):
            dnn_res = self.auxi_layers[i](dnn_res)
        return dnn_res + 0.00000001

In [63]:
class auxi_loss_layer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        self.dnn_part = auxiliary_net([100,50,2])
    def call(self, rnn_outputs,hist_item_emb,noclk_his_emb,item_mask):
        
#         print(rnn_outputs.shape,hist_item_emb.shape, noclk_his_emb.shape,item_mask.shape)
        # (128, 99, 64) (128, 99, 16) (128, 99, 16) (128, 99)  (B, SL-1, HD) 
        click_input = tf.concat([rnn_outputs, hist_item_emb],axis=-1)  # (128,99,80)
        noclick_input = tf.concat([rnn_outputs, noclk_his_emb], axis=-1)
         # 取出预测结果中点击的概率
        click_prop = self.dnn_part(click_input)[:,:,0] 
         # 取出预测结果中不点击的概率
        noclick_prop = self.dnn_part(noclick_input)[:,:,0] #  (128, 99, 2)-> (128, 99)
        # 计算loss
        click_loss = - tf.math.log(click_prop) * item_mask
        noclick_loss = -tf.math.log(1.0-noclick_prop) * item_mask
        loss_ = tf.reduce_mean(click_loss + noclick_loss)
        return loss_

In [64]:
class AUGRUCell(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        # RNN单元的必须参数，代表每个时间步的输出维度
        self.units = units
        self.state_size = units
    def build(self, input_shape):
        # t时刻 输入的 x_t的维度
        dim_xt = input_shape[0][-1]
        # 重置门对t时刻输入数据的权重
        self.W_R_x = tf.Variable(tf.random.normal(shape=[dim_xt, self.units]), name='W_R_x')
        # 重置门对t时刻的隐藏状态state的权重参数
        self.W_R_s = tf.Variable(tf.random.normal(shape=[self.units, self.units], name='W_R_s'))
        # 重置门的偏置
        self.W_R_b = tf.Variable(tf.random.normal(shape=[self.units]), name='W_R_b')
        
        # 更新门的对输入数据的权重
        self.W_U_x = tf.Variable(tf.random.normal(shape=[dim_xt, self.units]), name='W_U_x')
        # 更新门对t时刻的隐藏状态state的权重参数
        self.W_U_s = tf.Variable(tf.random.normal(shape=[self.units, self.units], name='W_U_s'))
        # 更新门的偏置
        self.W_U_b = tf.Variable(tf.random.normal(shape=[self.units]), name='W_U_b')
        
        # 计算当前时刻单元的状态: hat_h_t
        # 输入数据的权重
        self.W_H_x = tf.Variable(tf.random.normal(shape=[dim_xt, self.units]), name='W_H_x')
        # 上一个单元隐藏层的权重
        self.W_H_s = tf.Variable(tf.random.normal(shape=[self.units, self.units]), name='W_H_s')
        # 偏置
        self.W_H_b = tf.Variable(tf.random.normal(shape=[self.units]), name='W_H_b')  
    def call(self, inputs, states):
        x_t, att_score = inputs
        states = states[0]
        
        """
        x_t: shape=(B, 2D)
        states: hidden_state(t-1), shape=(B, units)
        att_score: attention_score(t), shape=(B,1)
        """
        # 重置门
        r_t = tf.sigmoid( tf.matmul(x_t,self.W_R_x) +  tf.matmul(states, self.W_R_s) + self.W_R_b ) # (B, units,)
        # 更新门
        u_t = tf.sigmoid(tf.matmul(x_t, self.W_U_x) + tf.matmul(states, self.W_U_s) + self.W_U_b) # (B, units,)
        # 带有注意力机制的更新门
        a_u_t = tf.multiply(att_score, u_t) # (B,1) * (B, units) = (B, units)
        # 当前时刻单元的状态: hat_h_t (B,units)
        hat_h_t = tf.tanh(tf.matmul(x_t, self.W_H_x)
                          + tf.matmul(tf.multiply(r_t, states) , self.W_H_s)
                          + self.W_H_b
                          )
        # 输出值
        h_t = tf.multiply(1-a_u_t, states) + tf.multiply(a_u_t, hat_h_t)
        # 对gru而言，当前时刻的output与传递给下一时刻的state相同
        next_state = h_t
        
        return h_t, next_state # 第一个表示output


In [65]:
class DIEN(tf.keras.Model):
    def __init__(self, user_count, item_count, cate_count, user_dim, item_dim, cate_dim,EMBEDDING_DIM, HIDDEN_SIZE, ATTENTION_SIZE, use_negsampling = False):
        super(DIEN,self).__init__()
        # 初始化用户、商品、类别 Embedding. l2正则化
        self.user_emb= layers.Embedding(user_count,user_dim,
                                              embeddings_regularizer=tf.keras.regularizers.l2(0.1),name='uid_embedding')
        self.item_emb = layers.Embedding(item_count,item_dim,
                                              embeddings_regularizer=tf.keras.regularizers.l2(0.1),name='item_embedding')
        self.cate_emb = layers.Embedding(cate_count,cate_dim,
                                              embeddings_regularizer=tf.keras.regularizers.l2(0.1),name='cat_embedding')
         # 构造模型的全连接层
        self.fc = fc_layer([128,60,2])
        self.gru = layers.GRU(64, return_sequences=True)
        self.auxi_loss = auxi_loss_layer()
        self.din_att = din_attention()
        
        self.AUGRU = layers.RNN(AUGRUCell(16)) # 初始化传入隐藏层的维度
  
    def get_emb(self,usr,item,cat,item_his, cat_his,noclk_mids, noclk_cats):
        user_emb = self.user_emb(usr)
        item_emb = self.item_emb(item)
        cat_emb = self.cate_emb(cat)
        item_his_emb = self.item_emb(item_his)
        cat_his_emb = self.item_emb(cat_his) 
        
        noclk_mids = self.item_emb(noclk_mids[:,:,1])
        noclk_cats = self.item_emb(noclk_cats[:,:,1])
        # 合并 concat
        new_item_emb = tf.concat([item_emb,cat_emb], axis=-1)
        new_item_his_emb = tf.concat([item_his_emb, cat_his_emb], axis=-1) 
        new_noclk_his_emb = tf.concat([noclk_mids, noclk_mids], axis=-1) 
        return user_emb,new_item_emb,new_item_his_emb,new_noclk_his_emb
    
    def call(self,usr,item,cat,item_his, cat_his,item_mask,sl,noclk_mids, noclk_cats):
        user_emb, item_emb, hist_item_emb,noclk_his_emb = self.get_emb(usr, item,cat, item_his,cat_his,noclk_mids, noclk_cats)
#         hist_item_emb_sum = tf.reduce_sum(hist_item_emb,axis=1)
        bool_mask = tf.cast(item_mask, tf.bool)
        gru_res = self.gru(hist_item_emb,mask=bool_mask)  #(B,SL,HD) 其中，SL表示sequence len, HD表示hidden Denmension
        auxi_loss = self.auxi_loss(gru_res[:,:-1,:],hist_item_emb[:,1:,:],noclk_his_emb[:,1:,:],item_mask[:,1:])
        attention_scores = self.din_att(item_emb, hist_item_emb,item_mask)  # (B, SL)
#         print(gru_res.shape,auxi_loss,attention_scores)
        
        augru_output = self.AUGRU((gru_res, attention_scores),mask=bool_mask) # (B,16)
        inp = tf.concat( [user_emb, item_emb,augru_output], axis=-1)
        out = self.fc(inp)
        y_hat = tf.nn.softmax(out) +  0.00000001
        return y_hat,auxi_loss
    
# (uids, mids, cats, mid_his, cat_his, mid_mask, sl,noclk_mids, noclk_cats)



In [66]:
din_model = DIEN(n_uid, n_mid, n_cat,8,8,8,EMBEDDING_DIM, HIDDEN_SIZE,ATTENTION_SIZE)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
i = 0  
for src, tgt in train_data:
    i += 1
    uids, mids, cats, mid_his, cat_his, mid_mask, target, sl, noclk_mids, noclk_cats = prepare_data(src, tgt, maxlen, return_neg=True)
    with tf.GradientTape() as tape:
        y_pred,auxi_loss = din_model(uids, mids, cats, mid_his, cat_his, mid_mask, sl,noclk_mids, noclk_cats)
        # loss = -tf.math.reduce_mean(tf.math.log(y_pred)*target)
        
        loss = tf.keras.losses.categorical_crossentropy(target, y_pred)
        loss = tf.reduce_mean(loss) + auxi_loss
        # print('batch %d: loss %f'%(i, loss.numpy()))
        if i % 10 == 0:
            print('batch %d: loss %f'%(i, loss.numpy()))
            if i== 50: break
    grads = tape.gradient(loss, din_model.trainable_variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, din_model.trainable_variables))


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/ddbb/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/backend.py", line 4198, in <genexpr>
    for ta, out in zip(output_ta_t, flat_new_output))  File "/home/ddbb/anaconda3/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py", line 237, in wrapped
    error_in_function=error_in_function)
batch 10: loss 2.005356
batch 20: loss 1.854979
batch 30: loss 1.990494
batch 40: loss 1.879137
batch 50: loss 1.950802


In [46]:
tf.cast([0,1], tf.bool)

<tf.Tensor: shape=(2,), dtype=bool, numpy=array([False,  True])>